In [1]:
%pwd
import os 
os.chdir(r"c:\vs\ml\chest-cancer-classification-using-mlflow-dvc")
%pwd

'c:\\vs\\ml\\chest-cancer-classification-using-mlflow-dvc'

In [2]:
import sys
from pathlib import Path

# Add project src directory to Python path
project_root = Path(r"c:\vs\ml\chest-cancer-classification-using-mlflow-dvc\src")
sys.path.append(str(project_root))

In [ ]:
#entity
from dataclasses import dataclass
from pathlib import Path
from typing import List
@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    validation_data: Path
    test_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list



In [15]:
import os
from pathlib import Path
from cnnClassifier.utils.common import read_yaml, create_directories
from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        
        # Updated paths for pre-split data
        base_dir = os.path.join(self.config.data_ingestion.unzip_dir, "Data")
        training_data = os.path.join(base_dir, "train")
        validation_data = os.path.join(base_dir, "valid")
        test_data = os.path.join(base_dir, "test")
        
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            validation_data=Path(validation_data),
            test_data=Path(test_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config



In [ ]:
#component
from pathlib import Path
import tensorflow as tf

class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        # Load the model architecture from the saved file
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
        
        # Build the model with a dummy input to initialize all layers
        dummy_input = tf.zeros((1, *self.config.params_image_size))
        _ = self.model(dummy_input)
        
        # Recompile with a fresh optimizer
        self.model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=['accuracy']
        )

    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        # Validation generator (no augmentation)
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.validation_data,
            shuffle=False,
            **dataflow_kwargs
        )

        # Training generator (with or without augmentation)
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            shuffle=True,
            **dataflow_kwargs
        )
        
        # Test generator (for evaluation after training)
        test_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        
        self.test_generator = test_datagenerator.flow_from_directory(
            directory=self.config.test_data,
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


In [17]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2025-10-04 02:47:40,256: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-04 02:47:40,262: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-04 02:47:40,266: INFO: common: created directory at: artifacts]
[2025-10-04 02:47:40,269: INFO: common: created directory at: artifacts\training]
[2025-10-04 02:47:40,638: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 72 images belonging to 4 classes.
Found 613 images belonging to 4 classes.
Found 315 images belonging to 4 classes.
Epoch 1/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 243s 6s/step - accuracy: 0.2630 - loss: 1.6846 - val_accuracy: 0.3594 - val_loss: 1.3881
Epoch 2/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 22s 457ms/step - accuracy: 0.3125 - loss: 1.3650 - val_accuracy: 0.2031 - val_loss: 1.3984
Epoch 3/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 183s 5s/step - accuracy: 0.2831 - loss: 1.3788 - val_